In [ ]:
#Parse the json file to DataFrame
import pandas as pd
import numpy as np
import gzip
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
from pyparsing import anyOpenTag, anyCloseTag
from xml.sax.saxutils import unescape as unescape
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
from collections import Counter
from scipy.sparse import csr_matrix


def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Health_and_Personal_Care_5.json.gz')
print df


In [ ]:
#Drop the columns not required
df.drop(['helpful','reviewerName','reviewerID','unixReviewTime','reviewTime'], axis=1)    

In [ ]:
#Add a new column avergarating int the dataframe.
df['averagerating'] = pd.Series(np.random.randn(346355), index=df.index)

In [ ]:
#Assign the data in the ratings column to the new column
df['averagerating'] = df['overall']

In [ ]:
#Aggregate the data based on product ID and the ratings and assign the average average rating to the new column 'avergarating'
df_final = df.groupby(['asin','overall']).agg({'averagerating':'mean', 
                        'reviewText': lambda x: ' , '.join(x),
                         'summary': lambda x: ' , '.join(x)}).reset_index()
                      #lambda x: "{%s}" % ', '.join(x))



In [ ]:
#convert the dataframe conatining reviewText and averagerating to the list
list_reviews = df_final['reviewText'].values.tolist()
list_ratings = df_final['averagerating'].values.tolist()

In [ ]:
#read thh file containing only the aggregated reviews
with open("aggregatedfile.dat", "r") as fr:
    f_reviews = fr.readlines()

In [ ]:
#removal of punctuation marks
import re
for i in range(0,60264):
      f_reviews[i] = re.sub('[^a-zA-Z0-9]', ' ', f_reviews[i])

In [ ]:
#split every word in the list containing the aggregated reviews
docs = [l.split() for l in f_reviews]

In [ ]:
#CSR Matrix
def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat

In [ ]:
#csr info of the csr matrix generated
def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )

In [ ]:
#build csr matrix
mat = build_matrix(docs)

csr_info(mat)

In [ ]:
#compute cs idf
def csr_idf(mat, copy=False, **kargs):
    r""" Scale a CSR matrix by idf. 
    Returns scaling factors as dict. If copy is True, 
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
        
    return df if copy is False else mat

In [ ]:
#compute L2 normalization
def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat

In [ ]:
mat2 = csr_idf(mat, copy=True)
mat3 = csr_l2normalize(mat2, copy=True)

In [ ]:
#divide the data into test and train
train_mat = mat3[:50000, :]
test_mat = mat3[50000:, :]

In [ ]:
#plot graph for test rating
train_rating = list_ratings[:50000]
test_rating = list_ratings[50000:50050]
test_rating1 = list_ratings[50000:]
X = list()
Y = list()
for i in range(0, len(test_rating)):
    X.append(i)
    Y.append(test_rating[i])
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(X,Y)



In [ ]:
#knn regression
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=20, weights='distance', algorithm='auto', leaf_size=30, p=3,
                            metric='euclidean', metric_params=None, n_jobs=10)
neigh.fit(train_mat, train_rating)
test_ratings_knn = neigh.predict(test_mat)

In [ ]:
#multiple linear regression
from sklearn.linear_model import LinearRegression
linearreg = LinearRegression(fit_intercept=False, normalize=False, copy_X=True, n_jobs=1)
linearreg.fit(train_mat, train_rating)
test_ratings_linear = linearreg.predict(test_mat)

In [ ]:
#Ridge regression
from sklearn.linear_model import Ridge
clf = Ridge(alpha=1.0)
clf.fit(train_mat, train_rating)
test_ratings_ridge = clf.predict(test_mat)

In [ ]:
#logistic regression
from sklearn.linear_model import LogisticRegression
regr = LogisticRegression()
regr.fit(train_mat, train_rating)
test_ratings_logistic = regr.predict(test_mat)

In [ ]:
#calculating mean square error
from sklearn.metrics import mean_squared_error
#mean_squared_error(test_true_rating, test_rating_predict, sample_weight=None, multioutput=’uniform_average’)
print mean_squared_error(test_rating1, test_ratings_knn, sample_weight=None)
print mean_squared_error(test_rating1, test_ratings_linear, sample_weight=None)
print mean_squared_error(test_rating1, test_ratings_ridge, sample_weight=None)
print mean_squared_error(test_rating1, test_ratings_logistic, sample_weight=None)


In [ ]:
#plot graph for test ratings
train_rating = list_ratings[:50000]
test_rating = list_ratings[50000:50050]
#print train_rating
#print test_rating
X = list()
Y = list()
for i in range(0, len(test_rating)):
    X.append(i)
    Y.append(test_rating[i])
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(X,Y)

In [ ]:
#plot graph for ratings predicted through ridge regression 
test_rating_ridge1 = test_ratings_ridge[0:50]
X = list()
Y = list()
for i in range(0, len(test_rating_ridge1)):
    X.append(i)
    Y.append(test_rating_ridge1[i])
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(X,Y)


In [ ]:
#plot graph for ratings predicted through logistic regression 
test_rating_logistic1 = test_ratings_logistic[0:50]
X = list()
Y = list()
for i in range(0, len(test_rating_logistic1)):
    X.append(i)
    Y.append(test_rating_logistic1[i])
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(X,Y)

In [ ]:
#plot graph for ratings predicted through knn
test_rating_knn1 = test_ratings_knn[0:50]
X = list()
Y = list()
for i in range(0, len(test_rating_knn1)):
    X.append(i)
    Y.append(test_rating_knn1[i])
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(X,Y)

In [ ]:
#plot graph for ratings predicted through linear regression
test_rating_linear1 = test_ratings_linear[0:50]
X = list()
Y = list()
for i in range(0, len(test_rating_linear1)):
    X.append(i)
    Y.append(test_rating_linear1[i])
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(X,Y)